In [139]:
import numpy as np
import pandas as pd

In [140]:
class Entite:
    def __init__(self, Lvl, PV, CA, F, Dex, Con, Int, Sag, Cha, nom =""):
        self.Lvl = Lvl
        self.PV = PV
        self.CA = CA
        self.F = F
        self.Dex = Dex
        self.Con = Con
        self.Int = Int
        self.Sag = Sag
        self.Cha = Cha
        self.f = self.modificateur(self.F)
        self.dex = self.modificateur(self.Dex)
        self.con = self.modificateur(self.Con)
        self.int = self.modificateur(self.Int)
        self.sag = self.modificateur(self.Sag)
        self.cha = self.modificateur(self.Cha)
        self.nom = nom

    def modificateur(self, caracteristique):
        return int(caracteristique/2 - 5)

class ActionCombatSimple:
    def __init__(self, nom, bonus_touche, degat):
        self.nom = nom
        self.bonus_touche = bonus_touche
        self.degat = degat
    def esperance(self, CA):
        delta = self.bonus_touche - CA + 10
        return (10+delta)/20 * self.degat
    def __repr__(self):
        return f"{self.nom}. Touche : {self.bonus_touche}, Degat : {self.degat}"

class ActionMultiple:
    def __init__(self, action_1, nombre_action_1, action_2 = "Rien", nombre_action_2 = 0):
        self.nom = "Multiple"
        self.action_1 = action_1
        self.nombre_action_1 = nombre_action_1
        self.action_2 = action_2
        self.nombre_action_2 = nombre_action_2

    def esperance(self, CA):
        return self.action_1.esperance(CA)*self.nombre_action_1 + self.action_2.esperance(CA)*self.nombre_action_2

    def __repr__(self):
        return f"Multiple. {self.action_1.nom} x {self.nombre_action_1}, {self.action_2.nom} x {self.nombre_action_2}"
        

class Monstre(Entite):
    def __init__(self, nom, Lvl, PV, CA, F, Dex, Con, Int, Sag, Cha, liste_action):
        Entite.__init__(self, Lvl, PV, CA, F, Dex, Con, Int, Sag, Cha, nom)
        self.actions = self.creation_actions(liste_action)

    def meilleure_esperance(self, CA):
        E = 0
        for nom, action in self.actions.items():
            if action.esperance(CA) > E:
                E = action.esperance(CA)
        return E
    def creation_action(self, action, dico):
        nom = action[0]
        if nom != "Multiple":
            nom, bonus_touche, degat = action
            return ActionCombatSimple(nom, bonus_touche, degat)
        else:
            nom, nom_action_1, nombre_action_1, nom_action_2, nombre_action_2 = action
            return ActionMultiple(dico[nom_action_1], nombre_action_1, dico[nom_action_2], nombre_action_2)

    def creation_actions(self, liste_action):
        dico = {}
        dico["Rien"] = ActionCombatSimple("Rien", 0, 0)
        for action in liste_action:
            dico[action[0]] = self.creation_action(action, dico)
        return dico

class Combat:
    def __init__(self, monstreA, monstreB):
        self.monstreA = monstreA
        self.monstreB = monstreB
        #self.pA = self.proba_toucher_and_degat(self.monstreA.actions, self.monstreB.CA)[0]
        #self.pB = self.proba_toucher_and_degat(self.monstreB.actions, self.monstreA.CA)[0]
        #self.dA = self.proba_toucher_and_degat(self.monstreA.actions, self.monstreB.CA)[1]
        #self.dB = self.proba_toucher_and_degat(self.monstreB.actions, self.monstreA.CA)[1]
        self.res = self.resultat()

    
    def resultat(self):
        setattr(self, "esperanceA", self.monstreA.meilleure_esperance(self.monstreB.CA))
        setattr(self, "esperanceB", self.monstreB.meilleure_esperance(self.monstreA.CA))
        a = self.esperanceA / self.esperanceB#self.pA * self.dA / (self.pB * self.dB)
        b = self.monstreB.PV - a * self.monstreA.PV
        nA = self.monstreA.PV / self.esperanceB #(self.pB * self.dB)
        nB = self.monstreB.PV / self.esperanceA #(self.pA * self.dA)
        if b <= 0:#vainqueur A ou égalité
            n = int(nB + 1)
            pv_A = self.monstreA.PV - n * self.esperanceB#self.pB * self.dB
            setattr(self, "n", n)
            setattr(self, "PV_restant", pv_A)
            setattr(self, "gagnant", self.monstreA)
            setattr(self, "perdant", self.monstreB)
            setattr(self, "degat_gagnant", self.esperanceA)
            setattr(self, "degat_perdant", self.esperanceB)
        else:
            n = int(nA + 1)
            pv_B = self.monstreB.PV - n * self.esperanceA #self.pA * self.dA
            setattr(self, "n", n)
            setattr(self, "PV_restant", pv_B)
            setattr(self, "gagnant", self.monstreA)
            setattr(self, "perdant", self.monstreB)
            setattr(self, "degat_gagnant", self.esperanceB)
            setattr(self, "degat_perdant", self.esperanceA)
    
    def __repr__(self):
        s = f"Durée : {self.n}. Vainqueur : {self.gagnant.nom}, PV restants : {self.PV_restant}, degat/tour : {self.degat_gagnant}, Perdant : {self.perdant.nom}, degat/tour : {self.degat_perdant}"
        return s



In [149]:
data = pd.read_csv('scrapper.csv')
list(data.iloc[0])[1:-3][-1]

"['Serres', 4, 4]"

In [133]:
cim = ActionCombatSimple("Cimeterre", 4, 5)
ActionMultiple(cim, 2, cim, 0)

Multiple. Cimeterre x 2, Cimeterre x 0

In [134]:
gobelin = Monstre("Gobelin", 1/4, 7, 15, 8, 14, 10, 10, 8, 8, [["Cimeterre", 4, 5], ["Multiple", "Cimeterre", 2, "Rien", 0]])
gobelin2 = Monstre("Gobelin2", 1/4, 7, 15, 8, 14, 10, 10, 8, 8, [["Cimeterre", 4, 5]])
grenouille = Monstre("Grenouille géante", 1/4, 18, 11, 12, 13, 11, 2, 10, 3, [["Morsure", 3, 4]])
homme_poisson = Monstre("Homme-poisson", 1/8, 11, 11, 10, 13, 12, 11, 11, 12, [["Lance", 2, 4]])
homme_lezard = Monstre("Homme-lézard", 1/2, 22, 15, 15, 10, 13, 7, 12, 7, [["Multipl", 4, 10]])
gargouille = Monstre("Gargouille", 2, 52, 15, 15, 11, 16, 6, 11, 7, [["Multipl", 4, 10]])

In [135]:
gobelin.meilleure_esperance(10)

7.0

In [138]:
c=Combat(gargouille, homme_poisson)
c

Durée : 2. Vainqueur : Gargouille, PV restants : 49.2, degat/tour : 6.5, Perdant : Homme-poisson, degat/tour : 1.4

In [85]:
homme_lezard.PV

22

In [86]:
4.5*5

22.5

In [16]:
c=Combat(gobelin, gargouille)
c

Vainqueur : Gargouille, PV restants : 47.5, durée : 2

In [44]:
u="rr"
print(type(u))

<class 'str'>
